In [ ]:
import torch
import torch.nn.functional as F
from torch import optim

from dataset import MetaDataset
from metadata.vcc import SanityMetadata

from models.vae import ToyVAE
from maml import reptile_train

In [ ]:
dataset = MetaDataset(SanityMetadata())
dataset.read_and_preprocess()

In [ ]:
device = torch.device('cpu')
model = ToyVAE()


def criterion(recon_y, y):
    BCE = F.binary_cross_entropy(recon_y, y, reduction='sum')
    # BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784),
    #                              reduce=True, size_average=False)
    # KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    # return BCE + KLD
    return BCE


optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=0.0)


def train(trained_model, x, y, n_inner_loop=4):
    trained_model.train()
    x = x.to(device)
    for _ in range(n_inner_loop):
        optimizer.zero_grad()
        out = trained_model(x)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()


reptile_train(model, dataset, n_shot=20, n_iter_outer=1000,
              outer_step_size=1e-2, inner_train_func=train)


